In [1]:
import os
import pandas as pd 
import numpy as np 
import sys
import glob
from Bio import SeqIO
import re

In [2]:
os.chdir('/fs/ess/PAS0439/MING/virome/amg_analysis/dram_annotation')

In [3]:
amg = pd.read_csv('amg_contig_individual_check.tsv', sep = '\t')

In [4]:
amg.contig_id = amg.contig_id.apply(lambda x: x.replace('__', '||'))

In [5]:
filtering_contigs = list(np.loadtxt('aj_filtered_contigs.txt', dtype='str'))

In [6]:
amg_filtering = amg[amg.contig_id.isin(filtering_contigs)]

In [7]:
amg_filtering.fillna('', inplace=True)

/tmp/slurmtmp.12160543/ipykernel_196179/2896627783.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amg_filtering.fillna('', inplace=True)


In [8]:
amg_filtering.loc[:, 'hits'] = amg_filtering['kegg_hit'] + amg_filtering['cazy_hits']  + amg_filtering['viral_hit']  + amg_filtering['pfam_hits'] + amg_filtering['vogdb_hit']

/tmp/slurmtmp.12160543/ipykernel_196179/2168348573.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amg_filtering.loc[:, 'hits'] = amg_filtering['kegg_hit'] + amg_filtering['cazy_hits']  + amg_filtering['viral_hit']  + amg_filtering['pfam_hits'] + amg_filtering['vogdb_hit']


In [9]:
#pd.set_option('display.max_rows', 700)
#gh_df = amg_filtering[amg_filtering.hits.str.contains('GH\d', regex=True)]
#gh_df_filter_virus = gh_df[~gh_df.amg_flags.str.contains('A|P')]
#gh_df_filter_virus_CBM_GH10_16 = gh_df_filter_virus[~gh_df_filter_virus.hits.str.contains('CBM|GH16|GH10 ')].query('contig_length < 200000')
#gh_containing= list(set(gh_df_filter_virus_CBM_GH10_16['contig_id']))

In [10]:
amg_filtering

,contig_id,contig_length,gene_position,kegg_hit,cazy_hits,viral_hit,pfam_hits,vogdb_hit,amg_flags,hits
0,SRR4435581_308300||full,178678,1,,,,,,F,
1,SRR4435581_308300||full,178678,2,,,,,,F,
2,SRR4435581_308300||full,178678,3,,,,,,F,
3,SRR4435581_308300||full,178678,4,,,YP_009910796.1 His-Xaa-Ser system radical SAM ...,Radical SAM superfamily [PF04055.24]; 4Fe-4S s...,REFSEQ His-Xaa-Ser system radical SAM maturase...,F,YP_009910796.1 His-Xaa-Ser system radical SAM ...
4,SRR4435581_308300||full,178678,5,,,,,,F,
...,...,...,...,...,...,...,...,...,...,...
331062,SRR12529379_935314||full,261996,236,,,,,,F,
331063,SRR12529379_935314||full,261996,237,,,,,,F,
331064,SRR12529379_935314||full,261996,238,,,,Protein of unknown function (DUF1653) [PF07866...,,F,Protein of unknown function (DUF1653) [PF07866...
331065,SRR12529379_935314||full,261996,239,,,YP_004934313.1 hypothetical protein phiR1-37_g...,Prohead core protein serine protease [PF03420.16],REFSEQ putative virion structural protein; Xu,F,YP_004934313.1 hypothetical protein phiR1-37_g...


In [11]:
LPS = ['carbohydrate kinase', 'carbohydrate-kinase', 'glycosyltransferase family 11 ', 'glycosyltransferase family 25', 'glycosyl transferase family 11 ', 'glycosyl transferase family 25', 'Glycosyltransferase family 11 ', 'Glycosyltransferase family 25', 'GT11 ', 'GT25', 'nucleotide sugar epimerase', 'nucleotide sugar-epimerase', 'nucleotide-sugar epimerase', 'nucleotide-sugar-epimerase', 'nucleotidyltransferase', 'nucleotidyl transferase', 'nucleotidyl-transferase', 'plasmid stability', 'endonuclease']

In [12]:
def filter_lps():
    furthur_check = []
    for index, row in amg_filtering.iterrows():
        hits = row['hits']
        votu = row['contig_id']
        if votu in furthur_check:
            continue
        else:
            if hits == '':
                continue
            else:
                for lps in LPS:

                    if lps in hits:
                        furthur_check.append(votu)
                    else:
                        continue
    
    return furthur_check
                    

In [13]:
furthur_check = filter_lps()

In [14]:
len(furthur_check)

1230

In [18]:
with open('/fs/ess/PAS0439/MING/virome/checkv_trimmed_for_dowmstream/AMG/supplemental_material/LPS_containing.txt', 'w') as outfile:
    for f in furthur_check:
        outfile.write(f + '\n')

In [19]:
amg_check = amg_filtering[amg_filtering.contig_id.isin(furthur_check)].sort_values('contig_length')

In [20]:
amg_check

,contig_id,contig_length,gene_position,kegg_hit,cazy_hits,viral_hit,pfam_hits,vogdb_hit,amg_flags,hits
255134,ERR4297088_90462||full,24893,18,uncharacterized protein,,YP_009613474.1 prohead protease [Enterococcus ...,Caudovirus prohead serine protease [PF04586.20],sp|D3WAC4|PRO_BPLP2 Probable capsid maturation...,VP,uncharacterized proteinYP_009613474.1 prohead ...
255119,ERR4297088_90462||full,24893,3,,,,,,F,
255120,ERR4297088_90462||full,24893,4,,,,,,F,
255121,ERR4297088_90462||full,24893,5,,,,Helix-turn-helix domain [PF14090.9],,F,Helix-turn-helix domain [PF14090.9]
255122,ERR4297088_90462||full,24893,6,,,,,,F,
...,...,...,...,...,...,...,...,...,...,...
21657,ERR3275110_431384||full,716757,1091,,,,,,,
21656,ERR3275110_431384||full,716757,1090,,,,,,,
21655,ERR3275110_431384||full,716757,1089,,,,,,,
21653,ERR3275110_431384||full,716757,1087,,,,,,,


In [25]:
amg_clean = amg_filtering[~amg_filtering.contig_id.isin(furthur_check)].sort_values('contig_length').reset_index(drop=True)

In [27]:
amg_clean.to_csv('/fs/ess/PAS0439/MING/virome/checkv_trimmed_for_dowmstream/AMG/supplemental_material/clean_amg_contigs_full_annotation.tsv', sep ='\t', index=None)

In [49]:
pd.set_option('display.max_rows', 900)

In [51]:
amg_clean_contigs = list(set(amg_clean['contig_id']))

In [52]:
len(amg_clean_contigs)

504

In [53]:
with open('/fs/ess/PAS0439/MING/virome/checkv_trimmed_for_dowmstream/AMG/clean_amg_contigs.txt', 'w') as outfile:
    for f in amg_clean_contigs:
        outfile.write(f + '\n')

In [31]:
pd.set_option('display.max_rows', 700)
gh_df = amg_clean[amg_clean.hits.str.contains('GH\d', regex=True)]
gh_df_filter_virus = gh_df[~gh_df.amg_flags.str.contains('A|P')]
gh_df_filter_virus_CBM = gh_df_filter_virus[~gh_df_filter_virus.hits.str.contains('CBM')]
gh_df_filter_virus_CBM
#gh_containing= list(set(gh_df_filter_virus_CBM_GH10_16['contig_id']))

,contig_id,contig_length,gene_position,kegg_hit,cazy_hits,viral_hit,pfam_hits,vogdb_hit,amg_flags,hits
267946,SRR948118_324373||full,41933,43,,"GH37 alpha,alpha-trehalase (EC 3.2.1.28). [GH3...",,Trehalase [PF01204.21]; Glycosyl hydrolase fam...,,M,"GH37 alpha,alpha-trehalase (EC 3.2.1.28). [GH3..."
138840,SRR11784289_330004||full,49990,85,,"GH166 alpha-1,4-galactosaminogalactan hydrolas...",,Glycoside-hydrolase family GH114 [PF03537.16],,F,"GH166 alpha-1,4-galactosaminogalactan hydrolas..."
290856,ERR2027913_1459896||full,52220,45,,"GH166 alpha-1,4-galactosaminogalactan hydrolas...",,Glycoside-hydrolase family GH114 [PF03537.16],,,"GH166 alpha-1,4-galactosaminogalactan hydrolas..."
281222,SRR11585831_164027||full,54136,25,,"GH166 alpha-1,4-galactosaminogalactan hydrolas...",,Glycoside-hydrolase family GH114 [PF03537.16],,,"GH166 alpha-1,4-galactosaminogalactan hydrolas..."
210083,SRR873595_1672480||full,55972,50,,"GH114 endo-alpha-1,4-polygalactosaminidase (EC...",,Glycoside-hydrolase family GH114 [PF03537.16],,M,"GH114 endo-alpha-1,4-polygalactosaminidase (EC..."
204601,SRR12529321_650355||full,55987,41,,"GH114 endo-alpha-1,4-polygalactosaminidase (EC...",,Glycoside-hydrolase family GH114 [PF03537.16],,M,"GH114 endo-alpha-1,4-polygalactosaminidase (EC..."
140856,ERR3201390_1008302||full,58792,110,,"GH166 alpha-1,4-galactosaminogalactan hydrolas...",,Glycoside-hydrolase family GH114 [PF03537.16],,F,"GH166 alpha-1,4-galactosaminogalactan hydrolas..."
287907,ERR4297095_788456||full,70125,79,,GH16 xyloglucan:xyloglucosyltransferase (EC 2....,,Glycosyl hydrolases family 16 [PF00722.24],,MKE,GH16 xyloglucan:xyloglucosyltransferase (EC 2....
154388,SRR873603_406764||full,72798,81,,"GH114 endo-alpha-1,4-polygalactosaminidase (EC...",,Glycoside-hydrolase family GH114 [PF03537.16];...,,M,"GH114 endo-alpha-1,4-polygalactosaminidase (EC..."
177213,ERR4297094_66449||full,75284,3,,GH74 endoglucanase (EC 3.2.1.4); oligoxylogluc...,,Photosynthesis system II assembly factor YCF48...,,MF,GH74 endoglucanase (EC 3.2.1.4); oligoxylogluc...
